In [2]:
import sqlite3
import random

conn = sqlite3.connect('movie_ticket.db')
cur = conn.cursor()
print("Connected to the database successfully")

#creating a table called MOVIE_DATA which will contain information(seats) on every film played 
cur.execute('''CREATE TABLE IF NOT EXISTS MOVIE_DATA (
	"ID"		INTEGER NOT NULL UNIQUE,
	"MOVIE"		TEXT NOT NULL,
	"CITY"		TEXT,
	"CENTER"	TEXT,
	"SCREEN"	INTEGER,
	"DATE"		TEXT,    
	"TIME"		TEXT,
	"SEATS"		INTEGER DEFAULT 30,
	PRIMARY KEY("ID" AUTOINCREMENT));''')
#once created intially it will not create a table once again if one already exists

#creating a table for the registered customers
cur.execute('''CREATE TABLE IF NOT EXISTS CUSTOMERS (
	"C_ID"		INTEGER NOT NULL UNIQUE,
	"NAME"		TEXT NOT NULL,
	"N_SEATS"	INTEGER NOT NULL,
	"M_ID"		INTEGER NOT NULL,
	PRIMARY KEY("C_ID" AUTOINCREMENT));''')



#checks if the particular record exists, if not creates a new one
def permutation(movie_name, place, cinema, screen, date, time):
    global cusname
    cur.execute('''SELECT * FROM MOVIE_DATA WHERE MOVIE = ? AND CITY = ? AND CENTER = ? AND SCREEN = ? AND DATE = ? AND TIME = ?''', (movie_name, place, cinema, screen, date, time))
    if cur.fetchall() != []:
        print()
    else:
        cur.execute('''INSERT OR IGNORE INTO MOVIE_DATA(MOVIE, CITY, CENTER, SCREEN, DATE, TIME, SEATS) VALUES(?, ?, ?, ?, ?, ?, ?)''', (movie_name, place, cinema, screen, date, time, random.randint(0, 30)))   
        conn.commit()
        print()

#checks the number of seats remaining at the location        
def checkseats(movie_name, place, cinema, screen, date, time):
    cur.execute('''SELECT SEATS FROM MOVIE_DATA WHERE MOVIE = ? AND CITY = ? AND CENTER = ? AND SCREEN = ? AND DATE = ? AND TIME = ?''', (movie_name, place, cinema, screen, date, time))
    N = cur.fetchone()[0]
    return N

def confirmbooking(movie_name, place, cinema, screen, date, time):
    global cusname
    remseats = checkseats(movie_name, place, cinema, screen, date, time) #getting the remaining seats
    print("Number of seats remaining :", remseats)
    ticket = int(input("How many tickets do you want?: "))
    print()
    ch = input("Do you wish to proceed (y/n) : ")
    print()
    if ch.lower() == 'y':
        #finish with the booking
        if remseats >= ticket:
            remseats -= ticket
            #update seats
            cur.execute('''UPDATE MOVIE_DATA SET SEATS = ? WHERE MOVIE = ? AND CITY = ? AND CENTER = ? AND SCREEN = ? AND DATE = ? AND TIME = ?;''', (remseats, movie_name, place, cinema, screen, date, time))
            #insert record into the customer table
            cur.execute('''SELECT ID FROM MOVIE_DATA WHERE MOVIE = ? AND CITY = ? AND CENTER = ? AND SCREEN = ? AND DATE = ? AND TIME = ?;''', (movie_name, place, cinema, screen, date, time))
            m_id = cur.fetchone()[0]
            #adding customer details
            cur.execute('''INSERT OR IGNORE INTO CUSTOMERS(NAME, N_SEATS, M_ID) VALUES(?, ?, ?);''', (cusname, ticket, m_id))
            conn.commit()
            print("Thank You", cusname, "\nSuccessful!!! Enjoy", movie_name, "at", cinema, place,"\non", date, time, "Screen :",screen, "Tickets Booked :", ticket)
        else:
            print("Not enough seats as required.\nSorry for the inconvinience", cusname)
    elif ch.lower() == 'n':
        print("Come again next time")
        return 
    else:
        print("Wrong Choice")
        return
    
#this t_movie function is used to select movie name
def t_movie(place, cinema):
    print("Which movie do you want to watch?")
    print("1, Avatar: The Way of Water")
    print("2, Spider-Man: Across the Spider-Verse")
    print("3, Oppenheimer")
    print("4, Mission: Impossible – Dead Reckoning")
    print("5, Top Gun: Maverick")
    m = int(input("Choose your movie: "))
    print()
    if m < 1 or m > 5:
        print("Wrong Choice")
        return
    if m == 1:
        datechoice(place, cinema, "Avatar: The Way of Water")
    elif m == 2:
        datechoice(place, cinema, "Spider-Man: Across the Spider-Verse")
    elif m == 3:
        datechoice(place, cinema, "Oppenheimer")
    elif m == 4:
        datechoice(place, cinema, "Mission: Impossible – Dead Reckoning")
    elif m == 5:
        datechoice(place, cinema, "Top Gun: Maverick")
        
# this theater function used to select screen
def theater(place, cinema, movie_name, date):
    print("Which screen do you want to watch the movie: ")
    print("1,SCREEN 1")
    print("2,SCREEN 2")
    print("3,SCREEN 3")
    a = int(input("Choose your screen: "))
    print()
    if a < 1 or a > 3:
        print("Wrong Choice")
        return
    timing(place, cinema, movie_name, a, date)
    
#this datechoice function is used to select the date of the movie
def datechoice(place, cinema, movie_name):
    print("On which date do you want to watch %s?" % movie_name)
    print("1: 16 / 2 / 2023")
    print("2: 17 / 2 / 2023")
    print("3: 18 / 2 / 2023")
    print("4: 19 / 2 / 2023")
    print("5: 20 / 2 / 2023")
    ch = int(input("Choose your date: "))
    print()
    if ch < 1 or ch > 5:
        print("Wrong Choice")
        return
    if ch == 1:
        theater(place, cinema, movie_name, "16 / 2 / 2023")
    elif ch == 2:
        theater(place, cinema, movie_name, "17 / 2 / 2023")
    elif ch == 3:
        theater(place, cinema, movie_name, "18 / 2 / 2023")
    elif ch == 4:
        theater(place, cinema, movie_name, "19 / 2 / 2023")
    elif ch == 5:
        theater(place, cinema, movie_name, "20 / 2 / 2023")

# this timing function used to select timing for movie
def timing(place, cinema, movie_name, screen, date):
    time1 = {
        "1": "10:00:00",
        "2": "1:10:00",
        "3": "4:20:00",
        "4": "7:30:00"
    }
    time2 = {
        "1": "10:15:00",
        "2": "1:25:00",
        "3": "4:35:00",
        "4": "7:45:00"
    }
    time3 = {
        "1": "10:30:00",
        "2": "1:40:00",
        "3": "4:50:00",
        "4": "8:00:00"
    }
    timechoice = [1, 2, 3, 4]
    if screen == 1:
        print("Choose your time :")
        print(time1)
        t = int(input("\nSelect your time : "))
        if t not in timechoice:
            print("Wrong Choice")
            return
        permutation(movie_name, place, cinema, screen, date, time1[str(t)]) #checking for the particular record in database
        confirmbooking(movie_name, place, cinema, screen, date, time1[str(t)]) #confirming the booking and updating data in database
    elif screen == 2:
        print("Choose your time :")
        print(time2)
        t = int(input("\nSelect your time : "))
        if t not in timechoice:
            print("Wrong Choice")
            return
        permutation(movie_name, place, cinema, screen, date, time2[str(t)])
        confirmbooking(movie_name, place, cinema, screen, date, time2[str(t)])
    elif screen == 3:
        print("Choose your time :")
        print(time3)
        t = int(input("\nSelect your time : "))
        if t not in timechoice:
            print("Wrong Choice")
            return
        permutation(movie_name, place, cinema, screen, date, time3[str(t)])
        confirmbooking(movie_name, place, cinema, screen, date, time3[str(t)])
    print("Thank you for visiting")
    return 0

# this function is used to select the theater
def center(place):
    print("In which theater do you wish to see the movie? ")
    print("1, IMAX")
    print("2, Cinepolis")
    print("3, PVR")
    a = int(input("Choose your option: "))
    print()
    if a == 1:
        t_movie(place, "IMAX")
    elif a == 2:
        t_movie(place, "Cinepolis")
    elif a == 3:
        t_movie(place, "PVR")
    else:
        print("Wrong Choice")
    return 0

# this function is used to select city
def city():
    print("\nWhere do you want to watch movie? : ")
    print("1, Bangalore")
    print("2, Mumbai")
    print("3, Delhi")
    place = int(input("Choose your option : "))
    print()
    if place == 1:
        center("Bangalore")
    elif place == 2:
        center("Mumbai")
    elif place == 3:
        center("Delhi")
    else:
        print("Wrong Choice")

def customerdata(name):
    ch = input("\nDo you want to check for ticket details (y/n) : ")
    print()
    if ch.lower() == 'y':
        #check for details
        cur.execute('''SELECT CUSTOMERS.NAME, MOVIE_DATA.MOVIE, CUSTOMERS.N_SEATS, MOVIE_DATA.CITY, MOVIE_DATA.CENTER, MOVIE_DATA.SCREEN, MOVIE_DATA.DATE, MOVIE_DATA.TIME 
                    FROM MOVIE_DATA JOIN CUSTOMERS 
                    ON MOVIE_DATA.ID = CUSTOMERS.M_ID
                    WHERE CUSTOMERS.NAME = ?;''', (name,))
        records = cur.fetchall()
        if records == []:
            print("No Records Found")
        else:
            for i in records:
                print("Name : "+i[0]+" || Movie : "+i[1]+" || Tickets Booked : "+str(i[2])+" || City : "+i[3]+"\n\t || Cinema : "+i[4]+" || Screen : ", str(i[5]), " || Date : "+i[6]," || Time : "+i[7]+" ||\n")
                
    else:
        return
    

print("Hello, Welcome to Movie Ticket Booking! ")
global cusname
cusname = input("Please enter your name : ")
customerdata(cusname) # it calls the customer records
city() # it calls the function city
conn.close() #closing connection to database

Connected to the database successfully
Hello, Welcome to Movie Ticket Booking! 
Please enter your name : Dhananjay

Do you want to check for ticket details (y/n) : y

No Records Found

Where do you want to watch movie? : 
1, Bangalore
2, Mumbai
3, Delhi
Choose your option : 2

In which theater do you wish to see the movie? 
1, IMAX
2, Cinepolis
3, PVR
Choose your option: 3

Which movie do you want to watch?
1, Avatar: The Way of Water
2, Spider-Man: Across the Spider-Verse
3, Oppenheimer
4, Mission: Impossible – Dead Reckoning
5, Top Gun: Maverick
Choose your movie: 2

On which date do you want to watch Spider-Man: Across the Spider-Verse?
1: 16 / 2 / 2023
2: 17 / 2 / 2023
3: 18 / 2 / 2023
4: 19 / 2 / 2023
5: 20 / 2 / 2023
Choose your date: 4

Which screen do you want to watch the movie: 
1,SCREEN 1
2,SCREEN 2
3,SCREEN 3
Choose your screen: 2

Choose your time :
{'1': '10:15:00', '2': '1:25:00', '3': '4:35:00', '4': '7:45:00'}

Select your time : 4

Number of seats remaining : 10
How m

## 